# 다양한 데이터 불러오기 방법

<br>


## 1. DB connection과 SQL 쿼리 실행(시연, 참고) 

일반적으로 **Database**를 활용할 때 SQL Client에 접속해서 **SQL query**를 실행합니다. 보안상 이슈가 있지만, Python에서 DB에 직접 연결한 다음 SQL query를 실행할 수 있습니다. **pandas**의 *read_sql()*를 활용하면 결과물을 바로 DataFrame 형식으로 받아와 활용할 수 있습니다.

In [1]:
# 라이브러리 설치
!pip install PyMySQL

     ---------------------------------------- 43.8/43.8 kB 2.1 MB/s eta 0:00:00


In [2]:
# connection 생성
    ## 강사 집에 임시로 만든 데이터베이스에 접속
import pandas as pd
import pymysql

conn = pymysql.connect(
    host="dataart.asuscomm.com",
    port = 3306, 
    user="test1",
    passwd="1234"
)

In [3]:
query = "SELECT * FROM sales.customers"
pd.read_sql(query, conn)

C:\Users\user\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000.0
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800.0
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300.0
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200.0
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,None,Philadelphia,PA,71270,USA,1323.0,72600.0
118,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,None,Brisbane,CA,94217,USA,1165.0,60300.0
119,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,None,London,None,WA1 1DP,UK,1501.0,43300.0
120,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,None,Boston,MA,51003,USA,1188.0,85100.0


<br>

## 2. API의 활용 및 json의 활용(시연, 참고)

API로 데이터를 받아올 수도 있습니다. API는 정해진 약속에 따라 데이터를 요청하는 방법으로 결과 데이터는 json 등의 형식으로 들어옵니다. json 형식은 적절한 방법으로 DataFrame 형식으로 바꿔 활용할 수 있습니다.  

기상청 API를 활용해서 **서초동**의 날씨 예보를 데이터로 가져와 보겠습니다.
 * 참고: [기상청 API](https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15084084)

In [4]:
# 기준 날짜, 기준 시간 설정
from datetime import datetime, timedelta
time1 = datetime.now()
bs_date = time1.strftime('%Y%m%d')
time2 = time1 - timedelta(minutes=12)
bs_time = ['0800', '1100', '1400', '1700'][(time2.hour>=11) + (time2.hour>=14) + (time2.hour>=17)]
bs_date, bs_time

('20230109', '1400')

In [5]:
# 기상청 
import requests

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst'
params ={'serviceKey':'1HcH5lG+R0QgBAxXB3jcBBOkcUbN61xgza6VbyQruMEVrTmgX1JQ9LsIeUFBHLG0/12ZmN5uKAfhPDMPjk+gaQ==', 
         'numOfRows':'1000', 
         'dataType':'JSON', # XML도 활용 가능
         'base_date':bs_date, 
         'base_time':bs_time, 
         'nx':'61', 
         'ny':'125'} 

response = requests.get(url, params=params)
print(response.content)

b'{"response":{"header":{"resultCode":"00","resultMsg":"NORMAL_SERVICE"},"body":{"dataType":"JSON","items":{"item":[{"baseDate":"20230109","baseTime":"1400","category":"TMP","fcstDate":"20230109","fcstTime":"1500","fcstValue":"6","nx":61,"ny":125},{"baseDate":"20230109","baseTime":"1400","category":"UUU","fcstDate":"20230109","fcstTime":"1500","fcstValue":"2.6","nx":61,"ny":125},{"baseDate":"20230109","baseTime":"1400","category":"VVV","fcstDate":"20230109","fcstTime":"1500","fcstValue":"-1.6","nx":61,"ny":125},{"baseDate":"20230109","baseTime":"1400","category":"VEC","fcstDate":"20230109","fcstTime":"1500","fcstValue":"302","nx":61,"ny":125},{"baseDate":"20230109","baseTime":"1400","category":"WSD","fcstDate":"20230109","fcstTime":"1500","fcstValue":"3.1","nx":61,"ny":125},{"baseDate":"20230109","baseTime":"1400","category":"SKY","fcstDate":"20230109","fcstTime":"1500","fcstValue":"1","nx":61,"ny":125},{"baseDate":"20230109","baseTime":"1400","category":"PTY","fcstDate":"20230109","fc

In [6]:
import json
json.loads(response.content)

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'baseDate': '20230109',
      'baseTime': '1400',
      'category': 'TMP',
      'fcstDate': '20230109',
      'fcstTime': '1500',
      'fcstValue': '6',
      'nx': 61,
      'ny': 125},
     {'baseDate': '20230109',
      'baseTime': '1400',
      'category': 'UUU',
      'fcstDate': '20230109',
      'fcstTime': '1500',
      'fcstValue': '2.6',
      'nx': 61,
      'ny': 125},
     {'baseDate': '20230109',
      'baseTime': '1400',
      'category': 'VVV',
      'fcstDate': '20230109',
      'fcstTime': '1500',
      'fcstValue': '-1.6',
      'nx': 61,
      'ny': 125},
     {'baseDate': '20230109',
      'baseTime': '1400',
      'category': 'VEC',
      'fcstDate': '20230109',
      'fcstTime': '1500',
      'fcstValue': '302',
      'nx': 61,
      'ny': 125},
     {'baseDate': '20230109',
      'baseTime': '1400',
      'category': 'WSD',
      '

In [7]:
# key 활용 부분 선택 
json.loads(response.content)['response']['body']['items']['item']

[{'baseDate': '20230109',
  'baseTime': '1400',
  'category': 'TMP',
  'fcstDate': '20230109',
  'fcstTime': '1500',
  'fcstValue': '6',
  'nx': 61,
  'ny': 125},
 {'baseDate': '20230109',
  'baseTime': '1400',
  'category': 'UUU',
  'fcstDate': '20230109',
  'fcstTime': '1500',
  'fcstValue': '2.6',
  'nx': 61,
  'ny': 125},
 {'baseDate': '20230109',
  'baseTime': '1400',
  'category': 'VVV',
  'fcstDate': '20230109',
  'fcstTime': '1500',
  'fcstValue': '-1.6',
  'nx': 61,
  'ny': 125},
 {'baseDate': '20230109',
  'baseTime': '1400',
  'category': 'VEC',
  'fcstDate': '20230109',
  'fcstTime': '1500',
  'fcstValue': '302',
  'nx': 61,
  'ny': 125},
 {'baseDate': '20230109',
  'baseTime': '1400',
  'category': 'WSD',
  'fcstDate': '20230109',
  'fcstTime': '1500',
  'fcstValue': '3.1',
  'nx': 61,
  'ny': 125},
 {'baseDate': '20230109',
  'baseTime': '1400',
  'category': 'SKY',
  'fcstDate': '20230109',
  'fcstTime': '1500',
  'fcstValue': '1',
  'nx': 61,
  'ny': 125},
 {'baseDate':

In [8]:
pd.DataFrame(json.loads(response.content)['response']['body']['items']['item'])

,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20230109,1400,TMP,20230109,1500,6,61,125
1,20230109,1400,UUU,20230109,1500,2.6,61,125
2,20230109,1400,VVV,20230109,1500,-1.6,61,125
3,20230109,1400,VEC,20230109,1500,302,61,125
4,20230109,1400,WSD,20230109,1500,3.1,61,125
...,...,...,...,...,...,...,...,...
695,20230109,1400,POP,20230112,0000,0,61,125
696,20230109,1400,WAV,20230112,0000,0,61,125
697,20230109,1400,PCP,20230112,0000,강수없음,61,125
698,20230109,1400,REH,20230112,0000,80,61,125


<br>

## 3.  BeautifulSoup을 활용한 웹스크랩/웹크롤링


*BeautifulSoup*은 네이버 뉴스처럼 별도의 로그인 절차 등이 없는 열린 페이지를 스크랩할 때 주로 활용합니다.  
<br> 



In [ ]:
# 라이브러리 설치
!pip install beautifulsoup4 lxml

In [ ]:
# 라이브러리 불러오기
from bs4 import BeautifulSoup
from urllib.request import urlopen


<br> 


## 3.1. 네이버 뉴스 스크랩 예제

네이버 등 포털사이트의 검색 결과 등을 간단히 스크랩 할 수 있습니다. 

In [ ]:
# 네이버 검색 후 URL 복사, 붙여넣기
url1 = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90'
page1 = urlopen(url1)
soup1 = BeautifulSoup(page1, 'html.parser')


In [ ]:
soup1

In [ ]:
# 클래스를 활용한 선택 
title_list = soup1.find_all(class_='news_tit')
title_list

In [ ]:
# 첫번째만 선택 후 구조 확인
title_list[0]

In [ ]:
# .attrs를 활용한 어트리뷰트 확인
title_list[0].attrs

In [ ]:
# 어트리뷰트를 활용한 선택
title_list[0]['href']

In [ ]:
title_list[0]['title']

In [ ]:
# for를 활용한 전체 기사의 제목만  리스트 형식으로 추출
# [x.get_text() for x in title_list]
[x['title'] for x in title_list]

<br>

for문을 활용해서 페이지를 바꿔가며 제목을 추가하는 것도 가능합니다.

In [ ]:
# 잠시멈춤을 위한 sleep 불러오기
from time import sleep

In [ ]:
# 빈 리스트 만들기
newstitle = list()

In [ ]:
# 웹사이트의 특성을 활용한 url 지정 후 스크랩
for i in range(0, 5):
    stt_value = str(1 + i*10)
    url_ = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90&start=' + stt_value
    page_ = urlopen(url_)
    soup_ = BeautifulSoup(page_, 'html.parser')
    title_list_ = soup_.find_all(class_='news_tit')
    
    newstitle.extend([x['title'] for x in title_list_])
    print('{}번째 페이지 스크랩 완료'.format(i+1))
    sleep(3) ## 3초 멈춤

In [ ]:
newstitle

In [ ]:
pd.DataFrame({'title':newstitle})

#### [참고] 하나의 파일로 저장 후 활용 

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import time
from datetime import datetime 

url1 = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90'
page1 = urlopen(url1)
soup1 = BeautifulSoup(page1, 'html.parser')

# 클래스를 활용한 선택 
title_list = soup1.find_all(class_='news_tit')
df_temp = pd.DataFrame({'title':[x['title'] for x in title_list]})

filename1 = datetime.now().strftime("title_%Y%m%d-%H%M%S.txt")
df_temp.to_csv(filename1)

In [ ]:
# EXE 만들기
# auto-py-to-exe


#### [참고] Selenium의 활용

[python selenium](https://www.google.com/search?q=python+selenium)
